In [7]:
#Compare process time when files are all distributed compared to just in 1 node?
#Same as untitled but will distribute the file to all nodes first and then executeimport findspark
#https://mapr.com/blog/tutorial-using-pyspark-and-mapr-sandbox/
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark

from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.sql import SQLContext

import numpy as np

from time import time

In [3]:
conf = pyspark.SparkConf()
conf.setAppName('ARBenchmark')
conf.set("spark.cores.max", "16")
conf.set("spark.yarn.executor.memoryOverhead", "0")
conf.set("spark.yarn.executor.memory", "512M")
conf.set("spark.yarn.driver.memory", "512M")
conf.set("spark.submit.deployMode", "client")
# sc.stop()
# sc = pyspark.SparkContext(conf=conf)
sc = pyspark.SparkContext.getOrCreate(conf=conf)

sqlContext = SQLContext(sc)

In [4]:
X_test = "ARBench/X_test.txt"
X_train = "ARBench/X_train.txt"
y_test = "ARBench/y_test.txt"
y_train = "ARBench/y_train.txt"

X_test_ = sc.textFile(X_test)
X_train_ = sc.textFile(X_train)
y_test_ = sc.textFile(y_test)
y_train_ = sc.textFile(y_train)

In [5]:
features = sc.textFile("ARBench/features.txt")
activity_labels = sc.textFile("ARBench/activity_labels.txt")

<h1>MLUTILS</h1>

In [6]:
features = sc.textFile('ARBench/X_train.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])
    
labels = sc.textFile('ARBench/y_train.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])

test_features = sc.textFile('ARBench/X_test.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])
    
test_labels = sc.textFile('ARBench/y_test.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])

In [8]:
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

#https://stackoverflow.com/questions/45002004/input-validation-failed-error-in-spark-mllib-multi-class-logistic-regression
#Labels shouls start at 0 
features = features.coalesce(1, False)
labels = labels.coalesce(1, False)

features_list = np.array(features.collect())
labels_list = np.array(labels.collect())

features = sc.parallelize(features_list, 5)
labels = sc.parallelize(labels_list, 5)

cols = ["label", "features"]
training = labels.zip(features).map(lambda lp: LabeledPoint(lp[0][0] - 1, lp[1:]))

test_features = features.coalesce(1, False)
test_labels = labels.coalesce(1, False)

test_features_list = np.array(features.collect())
test_labels_list = np.array(labels.collect())

test_features = sc.parallelize(test_features_list, 5)
test_labels = sc.parallelize(test_labels_list, 5)

test = test_labels.zip(test_features).map(lambda lp: LabeledPoint(lp[0][0] - 1, lp[1:]))
# print(test.take(10))

In [ ]:
start_train = time()

model = LogisticRegressionWithLBFGS.train(training, numClasses=12)

train_elapsed = time() - start_train
print(train_elapsed)

start_prediction_all = time()

predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

prediction_elapsed = time() - start_prediction_all
print(prediction_elapsed)

# Instantiate metrics object

start_metrics = time()

metrics = MulticlassMetrics(predictionAndLabels)
metrics_elapsed = time() - start_metrics
print(metrics_elapsed)

In [ ]:
# Overall statistics
accuracy = metrics.accuracy
recall = metrics.accuracy
f1Score = metrics.accuracy
print("Summary Stats")
print("Accuracy = %s" % accuracy)
# print("Recall = %s" % recall)
# print("F1 Score = %s" % f1Score)

# Statistics by class
labels = test.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))
    print("\n")

# Weighted stats
print("Weighted recall = %s" % metrics.weightedRecall)
print("Weighted precision = %s" % metrics.weightedPrecision)
print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)

<h1>ML</h1>

In [3]:
features = sc.textFile('ARBench/X_train.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])
    
labels = sc.textFile('ARBench/y_train.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])

cols = ["label", "features"]
training = labels.zip(features).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(cols)

print(features.mapPartitions(lambda iter: [sum(1 for _ in iter)]).collect())
print(labels.mapPartitions(lambda iter: [sum(1 for _ in iter)]).collect())

features = features.coalesce(1, False)
labels = labels.coalesce(1, False)

features_list = np.array(features.collect())
labels_list = np.array(labels.collect())

features = sc.parallelize(features_list, 5)
labels = sc.parallelize(labels_list, 5)
# dataRDD = images.zip(labels) 
training = labels.zip(features).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(cols)

[3884, 3883]
[3876, 3891]


In [4]:
# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())


LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw predi

In [5]:
features = sc.textFile('ARBench/X_test.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])
    
labels = sc.textFile('ARBench/y_test.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])

features = features.coalesce(1, False)
labels = labels.coalesce(1, False)

features_list = np.array(features.collect())
labels_list = np.array(labels.collect())

features = sc.parallelize(features_list, 5)
labels = sc.parallelize(labels_list, 5)

cols = ["label", "features"]
test = labels.zip(features).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(cols)

In [6]:
# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(test)

In [7]:
result = prediction.select("features", "label", "myProbability", "prediction") \
    .collect()

In [8]:
correct = 0
total = 0
for row in result[:]:
    if row.label == row.prediction:
#         print("same")
        correct = correct + 1
    else:
#         print("not same")
        pass
    total = total + 1
#     print("label=%s prediction=%s"
#           % (row.label, row.prediction))
print(correct)
print(total)

2745
3162


In [ ]:
print(correct)
print(total)

In [ ]:
from pyspark.ml.linalg import Vectors

features = sc.textFile("ARBench/test.txt")
labels = sc.textFile("ARBench/test_labels.txt")

llabels = [[0], [1], [0]]
rlabels = labels.collect()

rows = sc.textFile('ARBench/test_labels.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])

# from pyspark.mllib.linalg.distributed import RowMatrix
# mat = RowMatrix(rows)
print(type(rows))
print(rows.collect())

feat_rows = sc.textFile('ARBench/test.txt')\
    .map(lambda line: line.split(" "))\
    .map(lambda line: [float(val) for val in line])

print(feat_rows.collect())
# dd = [(labels[i][0], Vectors.dense(features[i])) for i in range(len(labels))]
# sc.parallelize(zip(labels, features)).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(["label", "features"])

In [ ]:
import numpy as np
cols = ["labels", "features"]
# z = rows.zip(feat_rows).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(cols)

z = rows.zip(feat_rows).map(lambda lp: (float(lp[0][0]), (lp[1:]))).toDF(cols)

input_data = [('retail', '2017-01-03T13:21:00'),
                 ('retail', '2017-01-03T13:21:00'),
                 ('retail', '2017-01-03T13:21:00')]
rdd_schema = StructType([StructField('business', StringType(), True), \
                     StructField('date', StringType(), True)])

# c = sqlContext.createDataFrame(input_data, rdd_schema)
# c.show()
# z.show()



crdd = z.rdd
print(crdd.take(1))
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

lp = crdd.map(lambda line: LabeledPoint(line[0], [line[1:]]))
print(lp.take(1))



# from pyspark.sql.functions import udf,col
# from pyspark.sql.types import IntegerType
# from pyspark import SparkContext
# from pyspark.sql import SQLContext

# function = udf(lambda col1 : col1, FloatType())
# newone = z.withColumn('col_n', function("labels"))

# print(newone.printSchema())
# print(newone.collect())
# print(newone.columns)

In [ ]:
import pyspark.sql.functions as func

df1 = sc.parallelize([('1234','13'),('6789','68')]).toDF(['col1','col2'])
df2 = sc.parallelize([('7777','66'),('8888','22')]).toDF(['col3','col4'])
df1.show()
# since there are no common column between these two dataframes add row_index so that it can be joined
df1=df1.withColumn('row_index', func.monotonically_increasing_id())
df2=df2.withColumn('row_index', func.monotonically_increasing_id())

# 'col3' from second dataframe (i.e. df2) is added to first dataframe (i.e. df1)
df1 = df1.join(df2["row_index","col4"], on=["row_index"]).drop("row_index")
df1.show()

cols = ["labels", "features"]
z = rows.zip(feat_rows).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(cols)
c = rows.zip(feat_rows).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(cols)

my_data =[('apple'), ('cat'),('none')]
from pyspark.sql import Row
R = Row('ID', 'words')
ff = sqlContext.createDataFrame([R(i, str(x)) for i, x in enumerate(my_data)])

# input_data = [('retail', '2017-01-03T13:21:00'),
#                  ('retail', '2017-01-03T13:21:00'),
#                  ('retail', '2017-01-03T13:21:00')]
# rdd_schema = StructType([StructField('business', StringType(), True), \
#                      StructField('date', StringType(), True)])

# c = sqlContext.createDataFrame(input_data, rdd_schema)

z = z.withColumn('row_index', func.monotonically_increasing_id())
ff = ff.withColumn('row_index', func.monotonically_increasing_id())
# z.show()
z = z.join(ff["row_index", "words"], on=["row_index"]).drop("row_index")
# z.count()
z.show()

In [ ]:
labels = [[0], [1], [0]]
features = [[2, 0, 0, 1], [0, 0, 0, 1], [0, 2, 0, 1]]
z = zip(labels, features)
for i in z:
    print(i)
    
t = sc.parallelize(zip(labels, features)).map(lambda lp: (float(lp[0][0]), Vectors.dense(lp[1]))).toDF(["label", "features"])
print(t.collect())

In [ ]:
print(x_test_df[1][2000])

In [ ]:
training = sqlContext.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

print(training.collect())

In [ ]:

# Prepare training data from a list of (label, features) tuples.
training = sqlContext.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())

# Prepare test data
test = sqlContext.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(test)
result = prediction.select("features", "label", "myProbability", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.myProbability, row.prediction))